In [1]:
def load_special_tokens(special_token_path, min_cnt):
    special_token_list, special_token_dict = [], {}
    with open(special_token_path, 'r', encoding = 'utf8') as i:
        lines = i.readlines()
        for l in lines:
            content_list = l.strip('\n').split()
            token = content_list[0]
            cnt = int(content_list[1])
            if cnt >= min_cnt:
                special_token_list.append(token)
                special_token_dict[token] = 1
    print (len(special_token_list))
    return special_token_list, special_token_dict

In [2]:
special_token_path = r'../data/totto_col_header_vocab.txt'
min_cnt = 10
special_token_list, special_token_dict = load_special_tokens(special_token_path, min_cnt)

2202


In [3]:
from transformers import BartTokenizerFast
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')
tokenizer.add_tokens(special_token_list)

from transformers import BartTokenizer
decode_tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')
decode_tokenizer.add_tokens(special_token_list)

2202

In [4]:
import torch
ckpt_path = r'./ckpt/finetune/generator-rl-finetune.ckpt'
model_ckpt = torch.load(ckpt_path, map_location='cpu')

In [5]:
from generator import Generator

In [6]:
model_name = r'facebook/bart-base'
tokenizer=decode_tokenizer
max_decode_len = 80
model = Generator(model_name, tokenizer, max_decode_len, dropout=0.0)

Initializing Huggingface BART model...
Resizing Token Embeddings...


In [7]:
model_parameters = model_ckpt['model']
model.load_state_dict(model_parameters)
model.eval()

Generator(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(52467, 768)
      (encoder): BartEncoder(
        (embed_tokens): Embedding(52467, 768)
        (embed_positions): LearnedPositionalEmbedding(1026, 768, padding_idx=1)
        (layers): ModuleList(
          (0): EncoderLayer(
            (self_attn): Attention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
            (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_af

In [8]:
from dataclass import *
import time
train_data_dict, dev_data_dict = {}, {}
train_data_dict['table_text_path'] = r'../data/tiny_data/tiny_table.txt'
train_data_dict['content_text_path'] = r'../data/tiny_data/tiny_content_plan.txt'
train_data_dict['reference_sentence_path'] = r'../data/tiny_data/tiny_reference.txt'
dev_data_dict['table_text_path'] = r'../data/tiny_data/tiny_table.txt'
dev_data_dict['content_text_path'] = r'../data/tiny_data/tiny_content_plan.txt'
dev_data_dict['reference_sentence_path'] = r'../data/tiny_data/tiny_reference.txt'
max_table_len = 320
max_content_plan_len = 25
max_tgt_len = 80
model_name = 'facebook/bart-base'
special_token_path = r'../data/totto_col_header_vocab.txt'
min_slot_key_cnt = 10
start_time = time.time()
use_RL = False
data = Data(train_data_dict, dev_data_dict, max_table_len, max_content_plan_len, max_tgt_len, 
        model_name, special_token_path, min_slot_key_cnt, use_RL)
print("--- %s seconds ---" % (time.time() - start_time))

Number of Special Token is 2202


original vocabulary Size 50265
vocabulary size after extension is 52467
50265
Start loading training data...
Loading Table Data...


100% |#######################################################################################################################################################################################################|


Loading Content Data...


100% |#######################################################################################################################################################################################################|


...

Loading Content Data...


100% |#######################################################################################################################################################################################################|


Loading Reference Data...


100% |#######################################################################################################################################################################################################|


Validation data loaded.
train number is 50, dev number is 50
--- 4.5783610343933105 seconds ---


In [9]:
table_path = r'../data/dev/totto_dev_table.txt'
content_path = r'../data/dev/totto_dev_content_plan.txt'
reference_path = r'../data/dev/totto_dev_reference.txt'

table_list, content_list, reference_list = [], [], []
with open(table_path, 'r', encoding = 'utf8') as i:
    lines = i.readlines()
    for l in lines:
        table_list.append(l.strip('\n'))

with open(content_path, 'r', encoding = 'utf8') as i:
    lines = i.readlines()
    for l in lines:
        content_list.append(l.strip('\n'))

with open(reference_path, 'r', encoding = 'utf8') as i:
    lines = i.readlines()
    for l in lines:
        reference_list.append(l.strip('\n'))

        
data_list = []
data_num = len(table_list)
for k in range(data_num):
    data_list.append((table_list[k], content_list[k], reference_list[k]))

# Generation with Reference Content Plan

In [10]:
idx = 11
table, content_plan, reference_sentence = data_list[idx]
print ('Table is:')
print (table + '\n')
print ('Reference Content Plan is:')
print (content_plan + '\n')
print ('Reference Sentence is:')
print (reference_sentence + '\n')
table_text = data_list[idx][0]
content_text = data_list[idx][1]
src_id_list = [data.sep_idx] + data.load_one_text_id(table_text, 320) + [data.sep_idx] + \
data.load_one_text_id(content_plan, 25)
batch_src_id_list = [src_id_list]
batch_src_tensor, batch_src_mask = data.process_source_tensor(batch_src_id_list)
result = model.generate(batch_src_tensor, batch_src_mask)[0]
print ('Generated Result is:')
print (result + '\n')

Table is:
__page_title__ : George Washington Colonials football __EOS__ __Date__ : 1956 __EOS__ __Bowl__ : Sun Bowl __EOS__ __Opponent__ : Texas Western __EOS__ __Result__ : W 13–0 __EOS__ __section_title__ : Bowl games __EOS__

Reference Content Plan is:
__Date__ __page_title__ __Result__ __Opponent__ __Bowl__

Reference Sentence is:
In 1956, George Washington Colonials scored 13–0 against Texas Western at the Sun Bowl.

Generated Result is:
In 1956, George Washington Colonials football team scored 13–0 against Texas Western in the Sun Bowl.



# Controlled Generation with Different Content Plan

In [11]:
idx = 11
table, _, reference_sentence = data_list[idx]
print ('Table is:')
print (table + '\n')
content_plan = '__Date__ __Result__ __Opponent__ __page_title__ __Bowl__'
print ('Input Content Plan is:')
print (content_plan + '\n')
print ('Reference Sentence is:')
print (reference_sentence + '\n')
table_text = data_list[idx][0]
content_text = data_list[idx][1]
src_id_list = [data.sep_idx] + data.load_one_text_id(table_text, 320) + [data.sep_idx] + \
data.load_one_text_id(content_plan, 25)
batch_src_id_list = [src_id_list]
batch_src_tensor, batch_src_mask = data.process_source_tensor(batch_src_id_list)
result = model.generate(batch_src_tensor, batch_src_mask)[0]
print ('Generated Result is:')
print (result + '\n')

Table is:
__page_title__ : George Washington Colonials football __EOS__ __Date__ : 1956 __EOS__ __Bowl__ : Sun Bowl __EOS__ __Opponent__ : Texas Western __EOS__ __Result__ : W 13–0 __EOS__ __section_title__ : Bowl games __EOS__

Input Content Plan is:
__Date__ __Result__ __Opponent__ __page_title__ __Bowl__

Reference Sentence is:
In 1956, George Washington Colonials scored 13–0 against Texas Western at the Sun Bowl.

Generated Result is:
In 1956, with a 13–0 victory over Texas Western, Colonials appeared in the Sun Bowl.



In [12]:
idx = 11
table, _, reference_sentence = data_list[idx]
print ('Table is:')
print (table + '\n')
content_plan = '__page_title__ __Opponent__ __Result__ __Date__ __Bowl__'
print ('Input Content Plan is:')
print (content_plan + '\n')
print ('Reference Sentence is:')
print (reference_sentence + '\n')
table_text = data_list[idx][0]
content_text = data_list[idx][1]
src_id_list = [data.sep_idx] + data.load_one_text_id(table_text, 320) + [data.sep_idx] + \
data.load_one_text_id(content_plan, 25)
batch_src_id_list = [src_id_list]
batch_src_tensor, batch_src_mask = data.process_source_tensor(batch_src_id_list)
result = model.generate(batch_src_tensor, batch_src_mask)[0]
print ('Generated Result is:')
print (result + '\n')

Table is:
__page_title__ : George Washington Colonials football __EOS__ __Date__ : 1956 __EOS__ __Bowl__ : Sun Bowl __EOS__ __Opponent__ : Texas Western __EOS__ __Result__ : W 13–0 __EOS__ __section_title__ : Bowl games __EOS__

Input Content Plan is:
__page_title__ __Opponent__ __Result__ __Date__ __Bowl__

Reference Sentence is:
In 1956, George Washington Colonials scored 13–0 against Texas Western at the Sun Bowl.

Generated Result is:
George Washington Colonials football defeated Texas Western by 13–0 in the 1956 Sun Bowl.



In [13]:
idx = 11
table, _, reference_sentence = data_list[idx]
print ('Table is:')
print (table + '\n')
content_plan = '__Bowl__ __page_title__ __Opponent__ __Result__ __Date__'
print ('Input Content Plan is:')
print (content_plan + '\n')
print ('Reference Sentence is:')
print (reference_sentence + '\n')
table_text = data_list[idx][0]
content_text = data_list[idx][1]
src_id_list = [data.sep_idx] + data.load_one_text_id(table_text, 320) + [data.sep_idx] + \
data.load_one_text_id(content_plan, 25)
batch_src_id_list = [src_id_list]
batch_src_tensor, batch_src_mask = data.process_source_tensor(batch_src_id_list)
result = model.generate(batch_src_tensor, batch_src_mask)[0]
print ('Generated Result is:')
print (result + '\n')

Table is:
__page_title__ : George Washington Colonials football __EOS__ __Date__ : 1956 __EOS__ __Bowl__ : Sun Bowl __EOS__ __Opponent__ : Texas Western __EOS__ __Result__ : W 13–0 __EOS__ __section_title__ : Bowl games __EOS__

Input Content Plan is:
__Bowl__ __page_title__ __Opponent__ __Result__ __Date__

Reference Sentence is:
In 1956, George Washington Colonials scored 13–0 against Texas Western at the Sun Bowl.

Generated Result is:
In the Sun Bowl, George Washington Colonials defeated Texas Western by 13–0 in 1956.

